In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Load
import pickle, gzip
with gzip.open('tokens10lakh.pkl.gz', 'rb') as f:
    tokens = pickle.load(f)

In [27]:
print(tokens[100:200])

[',', 'I', 'could', 'feel', 'humanity', ',', 'the', 'way', 'I', "'m", 'able', 'to', 'feel', 'my', 'body', '..', 'After', 'a', 'few', 'hundred', 'years', ',', 'the', 'pattern', 'became', 'obvious', ',', 'no', 'longer', 'the', 'war', 'and', 'damage', 'that', 'would', 'devastate', 'me', 'over', 'and', 'over', 'again', 'in', 'the', 'far', 'past', 'was', 'effecting', 'me', 'so', 'dominantly', '.', '<', 'newline', '>', 'It', "'s", 'funny', ',', 'but', 'I', 'felt', 'as', 'if', 'after', 'gaining', 'what', 'I', 'desired', 'so', 'long', ',', 'what', 'I', 'have', 'lived', 'for', 'my', 'entire', 'life', ',', 'only', 'then', ',', 'when', 'I', 'achieved', 'immortality', 'I', 'started', 'truly', 'aging', '.', '<', 'newline', '>', '<', 'newline', '>', '5', 'world']


In [21]:
len(tokens)

1000000

In [29]:
import string

# Sample list with punctuation marks represented as words
# words = ["hello", "world", ".", "!", "#", "python", ",", "newline", "?"]

# Define punctuation to remove (exclude hash #)
punctuation_to_remove = ''.join(c for c in string.punctuation if c != '#')

# Filter out punctuation (except #) and the word "newline"
filtered_words = [
    word for word in tokens
    if word not in punctuation_to_remove and word != "newline"
]

In [30]:
len(filtered_words)

766250

In [8]:
# import pickle, gzip
# # Save
# with gzip.open('tokens10lakh.pkl.gz', 'wb') as f:
#     pickle.dump(tokens, f)

In [5]:
%pip install tensorflow

In [33]:
import numpy as np
from nltk.tokenize import word_tokenize
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from tensorflow.keras.utils import to_categorical

In [34]:
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available: 1


In [36]:
tokens=filtered_words

In [38]:
len(tokens)

766250

In [37]:
len(set(tokens))

35899

In [47]:
# Step 1: Build Vocabulary
vocab = set(tokens)  # Unique words
V = len(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}
idx_to_word = {i: word for i, word in enumerate(vocab)}
print("Vocabulary size (V):", V)

# Step 2: Prepare Sequences
n = 5  # Context window size
X, y = [], []
for i in range(len(tokens) - n):
    X.append([word_to_idx[w] for w in tokens[i:i+n]])  # Input: n previous words
    y.append(word_to_idx[tokens[i+n]])                 # Target: next word
X = np.array(X)
y = np.array(y)
print("X shape:", X.shape, "y shape:", y.shape)

Vocabulary size (V): 35899
X shape: (766245, 5) y shape: (766245,)


In [51]:
# Step 3: Define MLP Model
embedding_dim = 10  # Smaller than 100 for this small example
model = Sequential([
    Embedding(input_dim=V, output_dim=embedding_dim, input_length=n),  # Embedding layer
    Flatten(),                                                        # Flatten to 1D
    Dense(64, activation='relu'),                                    # Hidden layer 1
    Dense(64, activation='relu'),                                     # Hidden layer 2
    Dense(V, activation='softmax')                                    # Output layer
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_4 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_4 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_12 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [42]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU Available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [52]:
# Step 4: Train the Model
with tf.device('/GPU:0'):
  model.fit(X, y, batch_size=128, epochs=5, validation_split=0.2)

Epoch 1/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 25s 5ms/step - accuracy: 0.0584 - loss: 7.2316 - val_accuracy: 0.1034 - val_loss: 6.5257
Epoch 2/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 38s 4ms/step - accuracy: 0.1095 - loss: 6.1716 - val_accuracy: 0.1158 - val_loss: 6.4126
Epoch 3/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.1241 - loss: 5.8580 - val_accuracy: 0.1195 - val_loss: 6.4389
Epoch 4/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 41s 4ms/step - accuracy: 0.1310 - loss: 5.6573 - val_accuracy: 0.1210 - val_loss: 6.5040
Epoch 5/5
4790/4790 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - accuracy: 0.1358 - loss: 5.5131 - val_accuracy: 0.1224 - val_loss: 6.6098


In [53]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [56]:
# Step 5: Generate Story
def generate_story(seed_text, model, n, word_to_idx, idx_to_word, max_len=100):
    seed_tokens = word_tokenize(seed_text.lower())
    if len(seed_tokens) < n:
        raise ValueError(f"Seed text must have at least {n} words")
    current_seq = [word_to_idx.get(w, 0) for w in seed_tokens[-n:]]  # Last n words

    story = seed_tokens.copy()
    for _ in range(max_len):
        input_seq = np.array([current_seq])
        pred = model.predict(input_seq, verbose=0)
        next_word_idx = np.argmax(pred[0])
        next_word = idx_to_word[next_word_idx]
        story.append(next_word)
        current_seq = current_seq[1:] + [next_word_idx]  # Shift window
    return " ".join(story)

# Test Generation
seed = "generate a good story or go to hell"
story = generate_story(seed, model, n, word_to_idx, idx_to_word)
print("Generated Story:", story)

Generated Story: generate a good story or go to hell I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I 'm sorry I
